# merge data as one csv for each training

## 1. duration = all

In [4]:
import pandas as pd
import os
import numpy as np
# train data
summary_folder_path = '../../datasets/train_data/summary'

games = { 'baseball':'/baseball/mlb',
         'iceball': '/iceball/nhl',
         'soccer_champion':'/soccer/champ_league',
         'scoccer_england':'/soccer/epl',
         'soccer_major':'/soccer/majorleague'}
# folder name is games, result name is games_results.csv

games_list = ['baseball','iceball']
soccer_list = ['soccer_champion','scoccer_england','soccer_major']

In [23]:
for game_name in games_list:
    training_df = pd.DataFrame()
    print(game_name)
    index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('duration')
        uncorrected = " ".join(map(str, round(series_df['home'],5)))
        global_corrected = " ".join(map(str, round(series_df['global_corrected'],5)))
        hourly_corrected = " ".join(map(str, round(series_df['hourly_corrected'],5)))
        daily_corrected = " ".join(map(str, round(series_df['daily_corrected'],5)))
        win = series_df.loc[0,'HOME_WIN']
        training_df = training_df.append({'no':str(index),
                            'game':game_name,
                            'uncorrected':uncorrected,
                            'global_corrected': global_corrected,
                            'hourly_corrected':hourly_corrected,
                            'daily_corrected':daily_corrected,
                            'win' : win},ignore_index=True)
        
#         break
    training_df.index.rename('index')
    training_df.to_csv('../../datasets/train_data/'+game_name+'.csv')

baseball


RangeIndex(start=0, stop=12061, step=1, name='index')

iceball


RangeIndex(start=0, stop=6949, step=1, name='index')

In [25]:
training_df = pd.DataFrame()
for game_name in soccer_list:   
    print(game_name)
    index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('duration')
        uncorrected = " ".join(map(str, round(series_df['home'],5)))
        global_corrected = " ".join(map(str, round(series_df['global_corrected'],5)))
        hourly_corrected = " ".join(map(str, round(series_df['hourly_corrected'],5)))
        daily_corrected = " ".join(map(str, round(series_df['daily_corrected'],5)))
        win = series_df.loc[0,'HOME_WIN']
        training_df = training_df.append({'no':str(index),
                            'game':game_name,
                            'uncorrected':uncorrected,
                            'global_corrected': global_corrected,
                            'hourly_corrected':hourly_corrected,
                            'daily_corrected':daily_corrected,
                            'win' : win},ignore_index=True)
        
training_df.index.rename('index')
training_df.to_csv('../../datasets/train_data/soccer.csv')

soccer_champion
scoccer_england
soccer_major


RangeIndex(start=0, stop=4295, step=1, name='index')

## 2.  separated by day

In [5]:
summary_folder_path = '../../datasets/train_data/summary'
mining_folder_path = '../../datasets/train_data/data_mining'
result_folder_path = '../../datasets/rawdata'
result_path = '../../datasets/results'

In [6]:
def direction_find(x):
    if x>0:
        return 1
    elif x==0:
        return 0
    else:
        return -1

In [21]:
def get_series(sec1,sec2,sec3,series_df,series_name):
    dic = {'sec1':" ".join(map(str, round(sec1[series_name],5))),
           'sec2':" ".join(map(str, round(sec2[series_name],5))),
           'sec3':" ".join(map(str, round(sec3[series_name],5))),
           'all':" ".join(map(str, round(series_df[series_name],5)))}
    
    return dic

unbiased = pd.DataFrame()
hourly_corrected = pd.DataFrame()
daily_corrected = pd.DataFrame()
global_corrected = pd.DataFrame()

for game_name in soccer_list:
    print(game_name)
    results = pd.read_csv(result_folder_path+games[game_name]+'_results.csv').set_index('Unnamed: 0')
    index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv')
        
        # sec 3
        sec3 = series_df[series_df['duration']<= int(series_df.iloc[-1,0]/3)] # sec 3
        # sec 2
        sec2 = series_df[(series_df['duration']<= int(series_df.iloc[-1,0]/3*2)) \
                  & (series_df['duration']> int(series_df.iloc[-1,0]/3)) ] #sec 2
        # sec 1
        sec1 = series_df[series_df['duration']> int(series_df.iloc[-1,0]/3*2)] # sec 1   
        if (len(sec1) < 1) or (len(sec2) < 1) or (len(sec3) < 1):
            continue
#         target1: result
        index = int(index)
        if results.loc[index,'home_score'] ==  results.loc[index,'away_score'] :
            win = {'draw':1,'home':0,'away':0}
        elif results.loc[index,'home_score'] >  results.loc[index,'away_score'] :
            win = {'draw':0,'home':1,'away':0}
        else:
            win = {'draw':0,'home':0,'away':1}
#        target2: price
        sec2_price = sec2.loc[sec2.index.tolist()[0],'home']
        sec3_price = sec3.loc[sec3.index.tolist()[0],'home']
        price = {'end_2':sec2_price,
                 'end_3':sec3_price}
#        target3: direction
        sec1_price = sec1.loc[sec1.index.tolist()[0],'home']
        direction = {'direction_2':direction_find(sec2_price-sec1_price),
                     'direction_3':direction_find(sec3_price-sec2_price)}

        unbiased_series = get_series(sec1,sec2,sec3,series_df,'home')
        hourly_corrected_series = get_series(sec1,sec2,sec3,series_df,'hourly_corrected')
        daily_corrected_series = get_series(sec1,sec2,sec3,series_df,'daily_corrected')
        global_corrected_series = get_series(sec1,sec2,sec3,series_df,'global_corrected')
        
        features = {**unbiased_series,**win,**price,**direction}
        unbiased = unbiased.append(features, ignore_index=True)
        features = {**hourly_corrected_series,**win,**price,**direction}
        hourly_corrected = hourly_corrected.append(features, ignore_index=True)
        features = {**daily_corrected_series,**win,**price,**direction}
        daily_corrected = daily_corrected.append(features, ignore_index=True)
        features = {**global_corrected_series,**win,**price,**direction}
        global_corrected = global_corrected.append(features, ignore_index=True)
unbiased.to_csv('../../datasets/train_data/series/soccer_unbiased.csv',index=False)
hourly_corrected.to_csv('../../datasets/train_data/series/soccer_hourly_corrected.csv',index=False)
daily_corrected.to_csv('../../datasets/train_data/series/soccer_daily_corrected.csv',index=False)
global_corrected.to_csv('../../datasets/train_data/series/soccer_global_corrected.csv',index=False)

soccer_champion
scoccer_england
soccer_major


In [22]:
for game_name in games_list:
    print(game_name)
    unbiased = pd.DataFrame()
    hourly_corrected = pd.DataFrame()
    daily_corrected = pd.DataFrame()
    global_corrected = pd.DataFrame()
    results = pd.read_csv(result_folder_path+games[game_name]+'_results.csv').set_index('Unnamed: 0')
    index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv')
        
        # sec 3
        sec3 = series_df[series_df['duration']<= int(series_df.iloc[-1,0]/3)] # sec 3
        # sec 2
        sec2 = series_df[(series_df['duration']<= int(series_df.iloc[-1,0]/3*2)) \
                  & (series_df['duration']> int(series_df.iloc[-1,0]/3)) ] #sec 2
        # sec 1
        sec1 = series_df[series_df['duration']> int(series_df.iloc[-1,0]/3*2)] # sec 1   
        if (len(sec1) < 1) or (len(sec2) < 1) or (len(sec3) < 1):
            continue
#         target1: result
        index = int(index)
        if results.loc[index,'home_score'] ==  results.loc[index,'away_score'] :
            win = {'draw':1,'home':0,'away':0}
        elif results.loc[index,'home_score'] >  results.loc[index,'away_score'] :
            win = {'draw':0,'home':1,'away':0}
        else:
            win = {'draw':0,'home':0,'away':1}
#        target2: price
        sec2_price = sec2.loc[sec2.index.tolist()[0],'home']
        sec3_price = sec3.loc[sec3.index.tolist()[0],'home']
        price = {'end_2':sec2_price,
                 'end_3':sec3_price}
#        target3: direction
        sec1_price = sec1.loc[sec1.index.tolist()[0],'home']
        direction = {'direction_2':direction_find(sec2_price-sec1_price),
                     'direction_3':direction_find(sec3_price-sec2_price)}

        unbiased_series = get_series(sec1,sec2,sec3,series_df,'home')
        hourly_corrected_series = get_series(sec1,sec2,sec3,series_df,'hourly_corrected')
        daily_corrected_series = get_series(sec1,sec2,sec3,series_df,'daily_corrected')
        global_corrected_series = get_series(sec1,sec2,sec3,series_df,'global_corrected')
        
        features = {**unbiased_series,**win,**price,**direction}
        unbiased = unbiased.append(features, ignore_index=True)
        features = {**hourly_corrected_series,**win,**price,**direction}
        hourly_corrected = hourly_corrected.append(features, ignore_index=True)
        features = {**daily_corrected_series,**win,**price,**direction}
        daily_corrected = daily_corrected.append(features, ignore_index=True)
        features = {**global_corrected_series,**win,**price,**direction}
        global_corrected = global_corrected.append(features, ignore_index=True)
    unbiased.to_csv('../../datasets/train_data/series/'+game_name+'_unbiased.csv',index=False)
    hourly_corrected.to_csv('../../datasets/train_data/series/'+game_name+'_hourly_corrected.csv',index=False)
    daily_corrected.to_csv('../../datasets/train_data/series/'+game_name+'_daily_corrected.csv',index=False)
    global_corrected.to_csv('../../datasets/train_data/series/'+game_name+'_global_corrected.csv',index=False)

baseball
iceball
